In [1]:
import random
seed_val = 1000
random.seed(seed_val)
import numpy as np
np.random.seed(seed_val)
import tensorflow as tf
tf.set_random_seed(seed_val)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten, Lambda
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras import regularizers
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
import gensim
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [2]:
def my_loss(y_true, y_pred):
    return y_pred

def neg_sample_loss(x):
    posZ, negZ = x
    posZ = K.log(K.sigmoid(posZ))
    negZ = K.log(K.sigmoid(-negZ))
    
    loss = -posZ - negZ
    return loss

def conv_model_multi(n_chars, n_consonant, n_vowels, n_units):
    target_word = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="target_word")
    context_word = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="context_word")
    negative_word = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="negative_word")
    
    input_word = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="input_word")
    t = Conv2D(10, (5, 5), padding='same', activation='relu')(input_word)
    t = MaxPooling2D(3, 3)(t)
    t = Flatten()(t)
    t = Dense(n_units, activation='linear')(t)
    target_model = Model(input_word, t, name="target_model")
    
    out_word = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="out_word")
    c = Conv2D(10, (5, 5), padding='same', activation='relu')(out_word)
    c = MaxPooling2D(3, 3)(c)
    c = Flatten()(c)
    c = Dense(n_units, activation='linear')(c)
    context_model = Model(out_word, c, name="context_model")
    
    target = target_model(target_word)
    context = context_model(context_word)
    negative = context_model(negative_word)
    posZ = Dot(axes=1, name="tar_dot_con")([target, context])
    negZ = Dot(axes=1, name="tar_dot_neg")([target, negative])
    
    loss_layer = Lambda(neg_sample_loss, name="neg_loss")([posZ, negZ])
    
    model = Model([target_word, context_word, negative_word], loss_layer)
    return model, target_model

In [ ]:

def generateSG(data, skip_window, batch_size,
			   int2word, char2tup, n_chars, n_consonant, n_vowels):
	win_size = skip_window  # np.random.randint(1, skip_window + 1)
	i = win_size
	while True:
		batch_input = []
		batch_output = []
		batch_vec_input = []
		for bi in range(0, batch_size, skip_window ):
			context = data[i - win_size: i + win_size + 1]
			target = context.pop(win_size)
			context = np.random.choice(context, skip_window, replace=False)
			targets = [target] * (win_size )
			batch_input.extend(targets)
			batch_output.extend(context)

			con_mat, vow_mat = word2vec_seperated(char2tup,
												  int2word[target], n_chars, n_consonant, n_vowels)
			word_mat = np.concatenate([con_mat, vow_mat], axis=1).reshape(
				(1, 1, n_chars, (n_consonant + n_vowels)))
			batch_vec_input.extend([word_mat] * (win_size))
			i += 1
			if i + win_size + 1 > len(data):
				i = win_size
		batch_vec_input = np.vstack(batch_vec_input)
		yield batch_input, batch_vec_input, batch_output


words = read_file("data/news.txt")[:2000]
words, word2freq = min_count_threshold(words)
# words = subsampling(words, 1e-3)
vocab, word2int, int2word = build_vocab(words)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
print("Words to train: ", len(words))
print("Vocabs to train: ", len(vocab))
#print("Unk count: ", word2freq['<unk>'])
int_words = words_to_ints(word2int, words)
int_words = np.array(int_words, dtype=np.int32)
n_chars = 11 + 2
n_epoch = 10
batch_size = 5
skip_window = 3
change = 5
init_lr = .05
gen = generateSG(list(int_words), skip_window, batch_size,
				 int2word, char2tup, n_chars, n_consonant, n_vowel)


In [ ]:
try:
    del train_model
    keras.backend.clear_session()
    gc.collect()
except:
    pass
train_model, target_model = conv_model_multi(n_chars, n_consonant, n_vowel, embed_size)
adam = keras.optimizers.Nadam(.0005)
train_model.compile(optimizer=adam, loss=my_loss)
gen = gen_imag_neg(list(int_words), skip_window, batch_size,
               int2word, char2tup,ns_unigrams, n_chars, n_consonant, n_vowel)

In [ ]:
filepath = "em/weight-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1)
callbacks = [checkpoint]
history = train_model.fit_generator(gen, steps_per_epoch=n_batches, epochs = 10, callbacks=callbacks)